In [10]:
"""Minimal example of a chain reader with chain reorganisation detection.

.. code-block:: shell

    python scripts/read-uniswap-v2-pairs-and-swaps-live.py

"""

import os
import time

from web3 import HTTPProvider, Web3

from eth_defi.abi import get_contract
from eth_defi.chain import install_chain_middleware
from eth_defi.event_reader.filter import Filter
from eth_defi.event_reader.reader import read_events, LogResult
from eth_defi.event_reader.reorganisation_monitor import JSONRPCReorganisationMonitor

In [11]:
json_rpc_url = os.environ.get("JSON_RPC_POLYGON", "https://polygon-rpc.com")
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()
install_chain_middleware(web3)

# Get contracts
Pair = get_contract(web3, "sushi/UniswapV2Pair.json")

filter = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)

# Get the headers of last 5 blocks before starting
reorg_mon.load_initial_block_headers(block_count=5)

processed_events = set()

latest_block = reorg_mon.get_last_block_live()

In [3]:
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
if chain_reorg_resolution.reorg_detected:
    print("Chain reorg warning")

evt: LogResult
for evt in read_events(
            web3,
            start_block=start,
            end_block=end,
            filter=filter,
):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]

    # The reader may cause duplicate events as the chain tip reorganises
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")

Swap at block:60,120,663 tx:0x43c3908fff8dccce8e3c2b0e773fcd932bccc0fe694a0c4d1947f7a9ce3961b0
Swap at block:60,120,664 tx:0x0c7b3795bc2297367e082a2c4e5d69c2e12ef6af54892747a73c005c861e5e65
Swap at block:60,120,667 tx:0xe5ca8851389377cfe7272e56185c967afd5856bf10ffd4159ede34d88bb02c35
Swap at block:60,120,668 tx:0xbacac2fbd34e955a76bcdfb88a89cc28c723d65eafbf54c59c47ca4b5ee64643
Swap at block:60,120,669 tx:0x5a50da5dbacacbdd8e3f20d1a3195f477d0c311fd49a6c584988215ac296e9bf
Swap at block:60,120,669 tx:0x32543763a4005e8c332dbd691029cb9b704f15a05cc105c65d348176c238a376
Swap at block:60,120,669 tx:0x967a3f4cee9c2cdadb7b61bdf762768837b2f06aa4d8e5a62642864709309f6a
Swap at block:60,120,669 tx:0x967a3f4cee9c2cdadb7b61bdf762768837b2f06aa4d8e5a62642864709309f6a
Swap at block:60,120,670 tx:0xd1a0aa4bfa3f60350bd81c9b1f927a433e930932bfb155964efbb1f58bba05d0
Swap at block:60,120,670 tx:0x743ff69b6e67ff75697a45da7374895aa6b1b48e7bd76434a6f16c1e0dfd5086
Swap at block:60,120,670 tx:0x743ff69b6e67ff75697a

In [7]:
evt

{'address': '0x2470e01e59a044184c8e4957827f79b1a051bfa4',
 'topics': ['0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822',
  '0x0000000000000000000000008308a7d7655c2ae21632ed3dd6f55f509e7dfdb9',
  '0x000000000000000000000000a0006e2f7973678e60a2e445ab8353abad6854d5'],
 'data': '0x0000000000000000000000000000000000000000000000039d1142b784754b470000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000005e0cd9ec92bc6c6440',
 'blockNumber': 60120671,
 'transactionHash': '0xaff07733ef087b0df76a0eaabe3586659bab249eb59f4ea3b028d7d12eb2cb74',
 'transactionIndex': '0x1f',
 'blockHash': '0x5295458f412b034845d025214b61a7703f93cf48c47983bb88a24aa839f8cb2d',
 'logIndex': '0x90',
 'removed': False,
 'context': None,
 'event': web3._utils.datatypes.Swap,
 'chunk_id': 60120662,
 'timestamp': 1722634926}

In [ ]:
# Keep reading events as they land
while True:
    chain_reorg_resolution = reorg_mon.update_chain()
    start, end = chain_reorg_resolution.get_read_range()

    if chain_reorg_resolution.reorg_detected:
        print("Chain reorg warning")

    evt: LogResult
    for evt in read_events(
            web3,
            start_block=start,
            end_block=end,
            filter=filter,
    ):
        # How to uniquely identify EVM logs
        key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]

        # The reader may cause duplicate events as the chain tip reorganises
        if key not in processed_events:
            print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
            processed_events.add(key)
    else:
        print(".")



    time.sleep(0.5)

In [ ]:
logging.basicConfig(level=os.environ.get("LOG_LEVEL", "info").upper(), handlers=[logging.StreamHandler()])

# Mute noise
logging.getLogger("web3.providers.HTTPProvider").setLevel(logging.WARNING)
logging.getLogger("web3.RequestManager").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.WARNING)

# HTTP 1.1 keep-alive
session = requests.Session()

json_rpc_url='https://mainnet.infura.io/v3/29547b52f77647b68da777e3ecc06811'
web3 = Web3(HTTPProvider(json_rpc_url, session=session))

# Enable faster ujson reads
patch_web3(web3)

web3.middleware_onion.clear()

install_chain_middleware(web3)

# Get contracts
Factory = get_contract(web3, "UniswapV2Factory.json")
Pair = get_contract(web3, "UniswapV2Pair.json")

In [ ]:
evt = read_events(
        web3,
        start_block=start,
        end_block=end,
        filter=filter, 
)

In [ ]:
decoded = decode_log(evt)

In [ ]:
decoded = decode_log(evt)
pair = fetch_pair_details_cached(web3, decoded["address"])
token0 = pair.token0
token1 = pair.token1
block_number = evt["blockNumber"]
print(block_number)

In [ ]:
evt: LogResult
for evt in read_events(
        web3,
        start_block=start,
        end_block=end,
        filter=filter, 
):
    decoded = decode_log(evt)
    pair = fetch_pair_details_cached(web3, decoded["address"])
    token0 = pair.token0
    token1 = pair.token1
    block_number = evt["blockNumber"]
    print(block_number)

In [ ]:
# Keep reading events as they land
while True:

    start = latest_block
    end = web3.eth.block_number

    evt: LogResult
    for evt in read_events(
        web3,
        start_block=start,
        end_block=end,
        filter=filter,
    ):

        decoded = decode_log(evt)

        # Swap() events are generated by UniswapV2Pool contracts
        pair = fetch_pair_details_cached(web3, decoded["address"])
        token0 = pair.token0
        token1 = pair.token1
        block_number = evt["blockNumber"]

        # Determine the human-readable order of token tickers
        if token0.symbol in QUOTE_TOKENS:
            base = token1  # token
            quote = token0  # stablecoin/BNB
            base_amount = decoded["args"]["amount1Out"] - decoded["args"]["amount1In"]
            quote_amount = decoded["args"]["amount0Out"] - decoded["args"]["amount0In"]
        else:
            base = token0  # stablecoin/BNB
            quote = token1  # token
            base_amount = decoded["args"]["amount0Out"] - decoded["args"]["amount0Out"]
            quote_amount = decoded["args"]["amount1Out"] - decoded["args"]["amount1Out"]

            # Calculate the price in Python Decimal class
            if base_amount and quote_amount:
                human_base_amount = base.convert_to_decimals(base_amount)
                human_quote_amount = quote.convert_to_decimals(quote_amount)
                price = human_quote_amount / human_base_amount

                if human_quote_amount > 0:
                    # We define selling when the stablecoin amount increases
                    # in the swap
                    direction = "sell"
                else:
                    direction = "buy"

                price = abs(price)

                print(f"Swap block:{block_number:,} tx:{evt['transactionHash']} {direction} price:{price:,.8f} {base.symbol}/{quote.symbol}")
            else:
                # Swap() event from DEX that is not Uniswap v2 compatible
                # print(f"Swap block:{block_number:,} tx:{evt['transactionHash']} could not decode")
                pass

    else:
        # No event detected between these blocks
        print(".")

    latest_block = end
    time.sleep(1)